In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

import os

In [15]:
def normalise_data(fname):
    data=pd.read_csv('../data/rnaseq_concealed/%s' % fname,sep=',',header=0,index_col=0)
    meta=pd.DataFrame(index=data.columns,columns=['Drug','Time','CC','Cell'])
    meta['Drug']=pd.Series(meta.index).apply(lambda x:x.split('_')[1]).values
    meta['Time']=pd.Series(meta.index).apply(lambda x:x.split('_')[3]).values
    meta['CC']=pd.Series(meta.index).apply(lambda x:x.split('_')[2]).values
    meta['Cell']=pd.Series(meta.index).apply(lambda x:x.split('_')[4]).values
    ctrl=meta.index[np.in1d(meta['Drug'],['dmso','untreated'])]
    norm_data=((data.T-data[ctrl].mean(1))/data[ctrl].std(1)).T
    meta.to_csv('../results/meta_conc/%s' % fname,sep=',')
    norm_data.to_csv('../results/norm_gex_conc/%s' % fname,sep=',')

In [16]:
fnames=os.listdir('../data/rnaseq_concealed/')
try:
    fnames.remove('.DS_Store')
except:
    pass

In [17]:
for fname in fnames:
    normalise_data(fname)

In [18]:
meta=pd.read_csv('../results/meta_conc/ASPC1-RNAseq-Perturbations.csv',sep=',',header=0,index_col=0)
data=pd.read_csv('../results/norm_gex_conc/ASPC1-RNAseq-Perturbations.csv',sep=',',header=0,index_col=0)

In [19]:
drugs=list(set(meta['Drug'])-set(['dmso','untreated']))
genes=list(data.index)

In [20]:
consensus=pd.DataFrame(index=genes,columns=drugs)

In [21]:
norm_data={}
meta_data={}
for fname in fnames:
    norm_data[fname]=pd.read_csv('../results/norm_gex_conc/%s' % fname,sep=',',header=0,index_col=0)
    meta_data[fname]=pd.read_csv('../results/meta_conc/%s' % fname,sep=',',header=0,index_col=0)

In [22]:
for drug in drugs:
    drug_data=pd.DataFrame(index=genes)
    for fname in fnames:
        indexes=meta_data[fname].index[meta_data[fname]['Drug']==drug]
        drug_data=pd.concat([drug_data,norm_data[fname][indexes]],1)
    drug_data[drug_data>10]=10
    drug_data[drug_data<-10]=-10
    consensus[drug]=drug_data.mean(1)

In [23]:
consensus.columns='cmpd_'+consensus.columns

In [24]:
consensus.to_csv('../results/consensus_conc.csv',sep=',')